# 📊 Exploração e Limpeza dos Dados
Este notebook tem como propósito realizar a **exploração inicial, pré-processamento e transformação dos dados** de nosso conjunto, garantindo sua qualidade para análises futuras.  
As etapas documentadas aqui incluem:

- Exploração das variáveis (estatística descritiva, gráficos e identificação de colunas numéricas e categóricas);  
- Tratamento de *missing values* e identificação/correção de *outliers*;  
- Transformações (codificação de variáveis categóricas e normalização de variáveis numéricas);  
- Registro de decisões que serão detalhadas na **Seção 4.2 da documentação**.  

Este material serve como evidência prática e complementa a documentação teórica escrita na seção 4.2 (*Compreensão dos Dados*) da Documentação.

## 0. Preparação do Ambiente

- Instalação e Importação das bibliotecas
- Carregamento da Nova Base de Dados


In [ ]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

# Verificação das versões das bibliotecas
print("=== VERIFICAÇÃO DAS BIBLIOTECAS ===")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print("✅ Bibliotecas importadas com sucesso!")

# Carregamento da Base de Dados com padrão de nomes
print("\n=== CARREGAMENTO DOS DADOS ===")
df_raw = pd.read_csv('../database/dataset inicial/nova_base_vendas_chillie.csv')
print(f"df_raw shape: {df_raw.shape}")

# Cópia para trabalho temporário
df = df_raw.copy()
print(f"df shape: {df.shape}")
print("✅ Dados carregados com sucesso!")

## 1. Exploração de Dados

Nesta seção, realizaremos uma análise inicial do conjunto de dados para compreender suas características principais. As etapas incluem:

- **Identificação de colunas numéricas e categóricas:** Classificação das variáveis para facilitar o tratamento e análise posterior.
- **Estatísticas descritivas:** Cálculo de métricas como média, mediana, desvio padrão, valores mínimos e máximos para variáveis numéricas.
Essas análises iniciais são fundamentais para orientar as etapas de pré-processamento e modelagem subsequentes.

### 1.1 Identificação de Colunas (Numéricas e Categóricas)

Nesta etapa, as colunas do dataset foram classificadas em diferentes categorias, como numéricas, categóricas, IDs e datas. Essa organização facilita o tratamento e a análise dos dados, permitindo aplicar estratégias específicas para cada tipo de variável.

In [ ]:
# Identificação e Padronização das Variáveis

# Colunas de identificação e CEPs (não participam de análises estatísticas)
ID_COLUMNS = [
    'ID_Loja', 'ID_Cliente', 'ID_Produto', 'ID_Deposito', 
    'Num_Vale', 'ID_SAP', 'Dim_Cliente.Cep_Cliente', 'Dim_Lojas.Cep_Emp'
]

# Colunas numéricas relacionadas a preços e valores monetários
NUMERIC_PRICE_COLUMNS = ['Preco_Custo', 'Valor_Total', 'Preco_Varejo', 'Desconto', 
                         'DESCONTO_CALCULADO', 'Total_Preco_Varejo', 'Total_Preco_Liquido']

# Outras colunas numéricas (quantidades, métricas, etc.)
NUMERIC_OTHER_COLUMNS = ['Quantidade']

# Colunas categóricas (texto/string que representam categorias)
CATEGORICAL_COLUMNS = [
    'Natureza_Operacao', 'Nome_Tabela_Preco', 'Dim_Cliente.Bairro_Cliente',
    'Dim_Cliente.Cidade_cliente', 'Dim_Cliente.Uf_Cliente', 'Dim_Cliente.Pais',
    'Dim_Cliente.Sexo', 'Dim_Cliente.Estado_Civil', 'Dim_Cliente.Regiao_Cliente',
    'Dim_Lojas.Nome_Emp', 'Dim_Lojas.Bairro_Emp', 'Dim_Lojas.Cidade_Emp',
    'Dim_Lojas.Estado_Emp', 'Dim_Lojas.Regiao', 'Dim_Lojas.Tipo_PDV',
    'Dim_Lojas.CANAL_VENDA', 'Dim_Lojas.SAP_NOME', 'Dim_Lojas.REGIAO_CHILLI',
    'Dim_Produtos.NomeDim_Produtos.Nome', 'Dim_Produtos.Grupo_Produto',
    'Dim_Produtos.Sub_Grupo', 'Dim_Produtos.Cor1', 'Dim_Produtos.Cor2',
    'Dim_Produtos.Material1', 'Dim_Produtos.Material2', 'Dim_Produtos.Segmentacao',
    'Dim_Produtos.Shape', 'Dim_Produtos.Formato', 'Dim_Produtos.Sexo',
    'Dim_Produtos.Griffe', 'Dim_Produtos.GRUPO_CHILLI'
]

# Colunas de data (para tratamento futuro específico)
DATE_COLUMNS = ['Dim_Cliente.Data_Cadastro', 'Dim_Cliente.Data_Nascimento']

print("✅ VARIÁVEIS DE TIPO PADRONIZADAS DEFINIDAS")
print(f"📋 IDs/CEPs: {len(ID_COLUMNS)} colunas")
print(f"💰 Preços/Valores: {len(NUMERIC_PRICE_COLUMNS)} colunas") 
print(f"🔢 Numéricas outras: {len(NUMERIC_OTHER_COLUMNS)} colunas")
print(f"📝 Categóricas: {len(CATEGORICAL_COLUMNS)} colunas")
print(f"📅 Datas: {len(DATE_COLUMNS)} colunas")
print(f"🎯 Total classificado: {len(ID_COLUMNS + NUMERIC_PRICE_COLUMNS + NUMERIC_OTHER_COLUMNS + CATEGORICAL_COLUMNS + DATE_COLUMNS)} colunas")

### 1.2 Estatísticas Básicas

Nesta seção, são realizadas análises descritivas das variáveis do conjunto de dados. As estatísticas básicas incluem:

- **Estatísticas descritivas para variáveis numéricas:** cálculo de métricas como média, mediana, desvio padrão, valores mínimos e máximos, além de percentis. Essas informações ajudam a compreender a distribuição e a variabilidade dos dados.
- **Estatísticas descritivas para variáveis categóricas:** identificação de valores únicos, moda e frequência de categorias, permitindo uma visão geral das características qualitativas do dataset.
- **Identificação de outliers:** análise de valores extremos nas variáveis numéricas utilizando o método do IQR (Intervalo Interquartílico), com cálculo de limites inferior e superior, além da proporção de outliers por variável.

Essas análises são fundamentais para identificar padrões, inconsistências e possíveis problemas nos dados, orientando as etapas subsequentes de limpeza e transformação.


In [ ]:

# Estatísticas Descritivas básicas das colunas numéricas
display(df.describe())

# Estatísticas Descritivas básicas das colunas categóricas
display(df.describe(include=['object', 'category']))


## 2. Pré-Processamento dos Dados

O pré-processamento é uma etapa essencial para garantir a qualidade dos dados antes da aplicação de modelos de aprendizado de máquina ou análises mais profundas.  
Nesta seção, realizamos ações que permitem tornar a base mais consistente, como a remoção de conulas irrelevantes para a análise, o tratamento de valores ausentes, a remoção ou correção de outliers, a normalização e a codificação de variáveis.  


#### 2.1 Exclusão de Colunas Não Relevantes

Nesta etapa, removemos colunas que não serão utilizadas em nossas análises ou transformações futuras.


In [ ]:
# 2.1 Exclusão de Colunas Irrelevantes
print("=== INFORMAÇÕES SOBRE VALORES NULOS (ANTES DA EXCLUSÃO) ===")
print(f"Total de valores nulos: {df.isnull().sum().sum()}")
print(f"Shape atual: {df.shape}")

# Colunas para serem excluídas
colunas_para_excluir = [
    'Frete',
    'Dim_Lojas.Data_Criacao_Emp',
    'ID_Faturamento',
    'ID_Vendedor',
    'ID_Date',
    'Documento',
    'DOC_UNICO',
    'Transacao',
    'Dt_update',
    'Dim_Cliente.Ativo',
    'Dim_Lojas.Cod_Franqueado',
    'Dim_Produtos.Cod_Auxiliar',
    'Dim_Produtos.Referencia',
    'Quantidade',
    'Preco_Varejo',
    'Num_Vale',
    'DESCONTO_CALCULADO',
    'ID_Deposito',
    'Dim_Cliente.Sexo',
    'Dim_Cliente.Data_Cadastro',
    'Dim_Cliente.Estado_Civil',
    'Dim_Produtos.Sub_Grupo',
    'Dim_Produtos.Cor1',
    'Dim_Produtos.Cor2',
    'Dim_Produtos.Material1',
    'Dim_Produtos.Material2',
    'Dim_Produtos.Segmentacao',
    'Dim_Produtos.Shape',
    'Dim_Produtos.Formato',
    'Dim_Produtos.Sexo',
    'Dim_Produtos.Griffe',
    'Natureza_Operacao',
    'Preco_Custo',
    'Valor_Total',
    'Nome_Tabela_Preco',
    'Dim_Cliente.Bairro_Cliente',
    'Dim_Cliente.Cep_Cliente',
    'Dim_Cliente.Cidade_cliente',
    'Dim_Cliente.Uf_Cliente',
    'Dim_Cliente.Pais',
    'Dim_Lojas.ID_SAP',
    'Dim_Lojas.SAP_NOME',
    'Dim_Lojas.Cep_Emp',
    'Dim_Produtos.NomeDim_Produtos.Nome'

]

# Verificar quais colunas existem antes de excluir
colunas_existentes_para_excluir = [col for col in colunas_para_excluir if col in df.columns]
colunas_inexistentes_para_excluir = [col for col in colunas_para_excluir if col not in df.columns]

print(f"\nColunas encontradas para exclusão: {len(colunas_existentes_para_excluir)}")
print(f"Colunas que serão excluídas: {colunas_existentes_para_excluir}")

if colunas_inexistentes_para_excluir:
    print(f"Colunas não encontradas (não serão excluídas): {colunas_inexistentes_para_excluir}")

# Excluir as colunas que existem
if colunas_existentes_para_excluir:
    df = df.drop(columns=colunas_existentes_para_excluir)
    print(f"\n✅ {len(colunas_existentes_para_excluir)} colunas foram excluídas com sucesso!")

print(f"\n=== RESULTADO DA EXCLUSÃO ===")
print(f"Shape após exclusão: {df.shape}")
print(f"Progressão: df_raw ({df_raw.shape}) -> df atual ({df.shape})")

### 2.2 Tratamento de Valores Ausentes  

Nesta etapa foram identificados e tratados os valores ausentes presentes no conjunto de dados.  
O processo teve como objetivo garantir a consistência da base e evitar distorções nas análises subsequentes.  

As ações realizadas incluíram:  

- **Identificação de variáveis com dados faltantes;**  
- **Avaliação da proporção de valores ausentes por coluna;**  
- **Definição de estratégias de tratamento,** que podem incluir remoção de registros incompletos ou imputação de valores com base em medidas estatísticas (média, mediana, moda) ou em critérios definidos pelo contexto da análise.  

O resultado é uma base de dados mais íntegra, reduzindo impactos negativos em processos de transformação, detecção de outliers e modelagem futura.  


In [ ]:
# 2.2 Tratamento de Valores Ausentes

print("=== TRATAMENTO DE VALORES AUSENTES ===")
print(f"Valores nulos antes do tratamento: {df.isnull().sum().sum()}")

# Identificar colunas categóricas (object ou category) e numéricas
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Preencher colunas categóricas com a moda (valor mais frequente)
for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        moda = df[col].mode(dropna=True)
        if not moda.empty:
            df[col] = df[col].fillna(moda[0])

# Preencher colunas numéricas com a mediana
for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        mediana = df[col].median()
        df[col] = df[col].fillna(mediana)

# Criar df_clean após remoção de colunas e imputação completa
df_clean = df.copy()

print(f"\n=== RESULTADO DO TRATAMENTO ===")
print(f"Valores nulos após tratamento: {df_clean.isnull().sum().sum()}")
print(f"✅ df_clean criado com sucesso!")
print(f"df_clean shape: {df_clean.shape}")
print(f"Progressão: df_raw ({df_raw.shape}) -> df_clean ({df_clean.shape})")

### 2.2.1 Filtros de Limpeza Avançada

Nesta etapa, aplicamos filtros específicos para limpar inconsistências nos dados:

- **Correção de idades inválidas:** Identificação e correção de datas de nascimento que resultem em idades menores que 18 anos ou maiores que 100 anos, substituindo-as pela idade média dos clientes válidos.
- **Remoção de registros com preços inválidos:** Exclusão de registros onde `Preco_Varejo` ou `Total_Preco_Varejo` sejam menores que 1, indicando dados inconsistentes.
- Remoção de registros que contenham "RELOGIO" na coluna "Dim_Produtos.GRUPO_CHILLI", indicando que produtos como relógios não são relevantes para a análise.

Estes filtros garantem maior qualidade e consistência dos dados para análises posteriores.



In [ ]:
import datetime

print("=== FILTROS DE LIMPEZA AVANÇADA ===")
print(f"Shape antes dos filtros: {df_clean.shape}")

# 1. Correção de Idades Inválidas
print("\n--- CORREÇÃO DE IDADES INVÁLIDAS ---")

# Verificar se a coluna de data de nascimento existe
if 'Dim_Cliente.Data_Nascimento' in df_clean.columns:
    # Converter para datetime
    df_clean['Dim_Cliente.Data_Nascimento'] = pd.to_datetime(df_clean['Dim_Cliente.Data_Nascimento'], errors='coerce')
    
    # Calcular idade atual
    hoje = datetime.datetime.now()
    df_clean['idade_calculada'] = (hoje - df_clean['Dim_Cliente.Data_Nascimento']).dt.days / 365.25
    
    # Identificar idades inválidas (< 18 ou > 100)
    idade_invalida = (df_clean['idade_calculada'] < 18) | (df_clean['idade_calculada'] > 100) | df_clean['idade_calculada'].isna()
    n_idades_invalidas = idade_invalida.sum()
    
    print(f"Idades inválidas encontradas: {n_idades_invalidas}")
    
    if n_idades_invalidas > 0:
        # Calcular idade média dos clientes válidos
        idades_validas = df_clean[~idade_invalida]['idade_calculada']
        idade_media = idades_validas.mean()
        
        # Calcular data de nascimento correspondente à idade média
        data_nascimento_media = hoje - datetime.timedelta(days=idade_media * 365.25)
        
        # Substituir datas inválidas pela data média
        df_clean.loc[idade_invalida, 'Dim_Cliente.Data_Nascimento'] = data_nascimento_media
        
        print(f"✅ {n_idades_invalidas} datas de nascimento corrigidas com idade média de {idade_media:.1f} anos")
    else:
        print("✅ Nenhuma idade inválida encontrada")
    
    # Remover coluna temporária
    df_clean = df_clean.drop('idade_calculada', axis=1)
else:
    print("⚠️ Coluna 'Dim_Cliente.Data_Nascimento' não encontrada")

# 2. Remoção de Registros com Preços Inválidos
print("\n--- REMOÇÃO DE PREÇOS INVÁLIDOS ---")

colunas_preco = ['Preco_Varejo', 'Total_Preco_Varejo']
colunas_preco_existentes = [col for col in colunas_preco if col in df_clean.columns]

if colunas_preco_existentes:
    # Identificar registros com preços baixos (< 1)
    mask_preco_baixo = pd.Series(False, index=df_clean.index)
    
    for col in colunas_preco_existentes:
        precos_baixos = (df_clean[col] < 1) | df_clean[col].isna()
        n_precos_baixos = precos_baixos.sum()
        print(f"Registros com {col} < 1: {n_precos_baixos}")
        mask_preco_baixo = mask_preco_baixo | precos_baixos
    
    # Aplicar filtro - manter apenas registros com preços válidos
    df_clean = df_clean[~mask_preco_baixo].reset_index(drop=True)
    
    print(f"✅ {mask_preco_baixo.sum()} registros removidos por preços inválidos")
else:
    print("⚠️ Colunas de preço não encontradas")

# Remoção de registros que contenham "RELOGIO e BAG" na coluna "Dim_Produtos.GRUPO_CHILLI"
if 'Dim_Produtos.GRUPO_CHILLI' in df_clean.columns:
    mask_relogio = df_clean['Dim_Produtos.GRUPO_CHILLI'].astype(str).str.upper().str.contains('RELOGIO|BAG', na=False)
    n_relogio = mask_relogio.sum()
    df_clean = df_clean[~mask_relogio].reset_index(drop=True)
    print(f"✅ {n_relogio} registros removidos contendo 'RELOGIO e BAG' em 'Dim_Produtos.GRUPO_CHILLI'")
else:
    print("⚠️ Coluna 'Dim_Produtos.GRUPO_CHILLI' não encontrada")

# Verificar duplicatas
print("\n--- VERIFICAÇÃO DE DUPLICATAS ---")
n_duplicatas = df_clean.duplicated().sum()
if n_duplicatas > 0:
    df_clean = df_clean.drop_duplicates().reset_index(drop=True)
    print(f"✅ {n_duplicatas} registros duplicados removidos")
else:
    print("✅ Nenhuma duplicata encontrada")

print(f"\n=== RESULTADO DOS FILTROS ===")
print(f"Shape após filtros: {df_clean.shape}")
print(f"Registros removidos: {df.shape[0] - df_clean.shape[0]}")
print(f"Progressão: df original ({df.shape}) -> df_clean filtrado ({df_clean.shape})")


### 2.2.2 Identificação e Tratamento de Outliers (IQR)

Nesta etapa, aplicamos o método do **IQR (Intervalo Interquartil)** para identificar e tratar outliers nas variáveis numéricas.

O processo consiste em:
- Calcular os quartis Q1 (25%) e Q3 (75%) para cada variável numérica
- Definir os limites inferior e superior usando a fórmula: Q1 - 1.5*IQR e Q3 + 1.5*IQR
- Identificar valores que estão fora destes limites como outliers
- Aplicar tratamento adequado (remoção ou winsorização) conforme a natureza da variável

**Importante:** Variáveis de identificação (IDs) não são consideradas nesta análise, pois não possuem significado numérico interpretável.

In [ ]:
# 2.2.2 Identificação e Tratamento de Outliers (IQR)

print("=== IDENTIFICAÇÃO E TRATAMENTO DE OUTLIERS (IQR) ===")
print(f"Shape antes do tratamento de outliers: {df_clean.shape}")

# Usar apenas variáveis numéricas relevantes (excluindo IDs)
numeric_cols_for_outliers = [col for col in NUMERIC_PRICE_COLUMNS + NUMERIC_OTHER_COLUMNS if col in df_clean.columns]
print(f"\nVariáveis numéricas para análise de outliers: {len(numeric_cols_for_outliers)}")
print(f"Colunas: {numeric_cols_for_outliers}")

# Função para calcular limites IQR
def calculate_iqr_bounds(series):
    """Calcula os limites inferior e superior usando o método IQR"""
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return lower_bound, upper_bound

# Análise de outliers por variável
outlier_analysis = []
total_outliers_removed = 0

print(f"\n--- ANÁLISE DE OUTLIERS POR VARIÁVEL ---")

for col in numeric_cols_for_outliers:
    # Obter valores válidos (não nulos)
    valid_values = df_clean[col].dropna()
    
    if len(valid_values) > 0:
        # Calcular limites IQR
        lower_bound, upper_bound = calculate_iqr_bounds(valid_values)
        
        # Identificar outliers
        outlier_mask = (df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)
        n_outliers = outlier_mask.sum()
        perc_outliers = (n_outliers / len(df_clean)) * 100
        
        # Armazenar informações
        outlier_info = {
            'variavel': col,
            'n_outliers': n_outliers,
            'perc_outliers': round(perc_outliers, 2),
            'limite_inferior': round(lower_bound, 2),
            'limite_superior': round(upper_bound, 2),
            'outlier_mask': outlier_mask
        }
        outlier_analysis.append(outlier_info)
        
        print(f"{col}:")
        print(f"  Outliers encontrados: {n_outliers} ({perc_outliers:.2f}%)")
        print(f"  Limites: [{lower_bound:.2f}, {upper_bound:.2f}]")

# Criar DataFrame com resumo dos outliers
outlier_summary = pd.DataFrame([
    {
        'Variável': info['variavel'],
        'N° Outliers': info['n_outliers'],
        '% Outliers': info['perc_outliers'],
        'Limite Inferior': info['limite_inferior'],
        'Limite Superior': info['limite_superior']
    }
    for info in outlier_analysis
])

print(f"\n--- RESUMO DE OUTLIERS ---")
outliers_found = outlier_summary[outlier_summary['N° Outliers'] > 0].sort_values('N° Outliers', ascending=False)

if len(outliers_found) > 0:
    print(f"Variáveis com outliers: {len(outliers_found)}")
    display(outliers_found)
    
    # Tratamento de outliers - Remoção de registros com outliers extremos
    print(f"\n--- TRATAMENTO DE OUTLIERS ---")
    
    # Identificar registros que são outliers em múltiplas variáveis (mais propensos a serem erros)
    combined_outlier_mask = pd.Series(False, index=df_clean.index)
    
    # Considerar para remoção apenas outliers em variáveis de preço/valor
    price_outlier_vars = [info for info in outlier_analysis 
                         if info['variavel'] in NUMERIC_PRICE_COLUMNS and info['n_outliers'] > 0]
    
    # Aplicar remoção conservadora - apenas casos extremos
    for info in price_outlier_vars:
        # Remover apenas outliers extremos (além de 3*IQR)
        col = info['variavel']
        series = df_clean[col].dropna()
        q1, q3 = series.quantile(0.25), series.quantile(0.75)
        iqr = q3 - q1
        extreme_lower = q1 - 3 * iqr
        extreme_upper = q3 + 3 * iqr
        
        extreme_outliers = (df_clean[col] < extreme_lower) | (df_clean[col] > extreme_upper)
        combined_outlier_mask = combined_outlier_mask | extreme_outliers
        
        if extreme_outliers.sum() > 0:
            print(f"{col}: {extreme_outliers.sum()} outliers extremos identificados para remoção")
    
    # Aplicar remoção
    if combined_outlier_mask.sum() > 0:
        df_clean = df_clean[~combined_outlier_mask].reset_index(drop=True)
        total_outliers_removed = combined_outlier_mask.sum()
        print(f"\n✅ {total_outliers_removed} registros com outliers extremos removidos")
    else:
        print("\n✅ Nenhum outlier extremo identificado para remoção")
        
else:
    print("✅ Nenhum outlier detectado nas variáveis numéricas!")

print(f"\n=== RESULTADO DO TRATAMENTO DE OUTLIERS ===")
print(f"Registros removidos por outliers extremos: {total_outliers_removed}")
print(f"Shape final após tratamento: {df_clean.shape}")
print(f"Progressão total de limpeza: {df_raw.shape[0] - df_clean.shape[0]} registros removidos ({((df_raw.shape[0] - df_clean.shape[0]) / df_raw.shape[0] * 100):.1f}%)")

### 2.3 Transformações das Variáveis

Após a limpeza inicial da base de dados, é necessário aplicar transformações nas variáveis para padronizar seus formatos e possibilitar análises e modelagens mais consistentes.  
Nesta seção, foram realizados os seguintes procedimentos: normalização de variáveis numéricas e codificação de variáveis categóricas.  


In [ ]:
print("=== USANDO VARIÁVEIS DE TIPO DEFINIDAS NO INÍCIO ===")
print(f"IDs/CEPs: {len(ID_COLUMNS)} colunas")
print(f"Preços/Valores: {len(NUMERIC_PRICE_COLUMNS)} colunas") 
print(f"Numéricas outras: {len(NUMERIC_OTHER_COLUMNS)} colunas")
print(f"Categóricas: {len(CATEGORICAL_COLUMNS)} colunas")
print(f"Datas (p/ depois): {len(DATE_COLUMNS)} colunas")

In [ ]:
# Criando a Classe de Tranformação
class DataTransformer:
    def __init__(self, df):
        self.df = df.copy()
        self.encoders = {}
        self.scalers = {}
        self.transformations_applied = {}

    def clean_categorical_data(self):
        """Padroniza strings e substitui valores problemáticos"""
        df_clean = self.df.copy()
        for col in CATEGORICAL_COLUMNS:
            if col in df_clean.columns:
                df_clean[col] = df_clean[col].astype(str).str.strip().str.upper()
                df_clean[col] = df_clean[col].replace(['NAN','NULL','NONE','','NaT'], np.nan)
        self.transformations_applied['categorical_cleaning'] = CATEGORICAL_COLUMNS
        return df_clean

    def encode_categorical_variables(self, df_clean):
        """Aplica encoding categórico"""
        df_encoded = df_clean.copy()
        for col in CATEGORICAL_COLUMNS:
            if col in df_encoded.columns:
                unique_count = df_encoded[col].nunique()
                if unique_count <= 10:
                    dummies = pd.get_dummies(df_encoded[col], prefix=f"{col}_", drop_first=True)
                    df_encoded = pd.concat([df_encoded.drop(col, axis=1), dummies], axis=1)
                    self.encoders[col] = 'one_hot'
                else:
                    le = LabelEncoder()
                    df_encoded[f"{col}_encoded"] = le.fit_transform(df_encoded[col])
                    df_encoded = df_encoded.drop(col, axis=1)
                    self.encoders[col] = le
        self.transformations_applied['encoding'] = CATEGORICAL_COLUMNS
        return df_encoded

    def scale_numeric_variables(self, df_encoded):
        """Aplica normalização nas variáveis numéricas"""
        df_scaled = df_encoded.copy()
        for col in NUMERIC_PRICE_COLUMNS:
            if col in df_scaled.columns:
                scaler = MinMaxScaler()
                df_scaled[col] = scaler.fit_transform(df_scaled[[col]])
                self.scalers[col] = scaler
        for col in NUMERIC_OTHER_COLUMNS:
            if col in df_scaled.columns:
                scaler = StandardScaler()
                df_scaled[col] = scaler.fit_transform(df_scaled[[col]])
                self.scalers[col] = scaler
        self.transformations_applied['scaling'] = NUMERIC_PRICE_COLUMNS + NUMERIC_OTHER_COLUMNS
        return df_scaled

    def transform_all(self):
        df_clean = self.clean_categorical_data()
        df_encoded = self.encode_categorical_variables(df_clean)
        df_scaled = self.scale_numeric_variables(df_encoded)
        return df_scaled


In [ ]:
# Aplicando as Transformações usando df_clean como entrada
print("=== APLICANDO TRANSFORMAÇÕES ===")
print(f"Entrada: df_clean shape = {df_clean.shape}")

transformer = DataTransformer(df_clean)
df_transformed = transformer.transform_all()

print(f"\n=== RESULTADO DAS TRANSFORMAÇÕES ===")
print(f"Transformações aplicadas: {list(transformer.transformations_applied.keys())}")
print(f"df_transformed shape: {df_transformed.shape}")

# Resumo da progressão completa
print(f"\n=== PROGRESSÃO COMPLETA DOS DATAFRAMES ===")
print(f"df_raw:         {df_raw.shape}")
print(f"df_clean:       {df_clean.shape}")
print(f"df_transformed: {df_transformed.shape}")
print(f"Colunas adicionadas: {df_transformed.shape[1] - df_clean.shape[1]}")

print(f"\n=== VERIFICAÇÕES DE QUALIDADE ===")
print(f"Tipos de dados:")
print(df_transformed.dtypes.value_counts())

null_counts = df_transformed.isnull().sum()
if null_counts.sum() > 0:
    print(f"\n⚠️ VALORES NULOS RESTANTES:")
    print(null_counts[null_counts > 0])
else:
    print("\n✅ Nenhum valor nulo restante!")

print(f"\n=== AMOSTRA DO RESULTADO FINAL ===")
df_transformed.head(5)

### 2.4 Salvamento das Bases de Dados  

Após as etapas de pré-processamento realizadas (remoção de colunas irrelevantes, tratamento de valores ausentes, filtros de limpeza avançada e transformações das variáveis), salvamos duas versões da base de dados:

1. **Dataset Limpo (`dataset_limpo.csv`):** Contém os dados após a limpeza e filtros, mas **antes das transformações** (normalização e codificação). Esta versão preserva os dados em formato original e é ideal para análises exploratórias e visualizações.

2. **Dataset Codificado (`dataset_codificado.csv`):** Contém os dados **após todas as transformações** aplicadas (normalização de variáveis numéricas e codificação de variáveis categóricas). Esta versão é otimizada para algoritmos de Machine Learning e modelagem estatística.

Ambas as versões incluem os filtros de qualidade aplicados: correção de idades inválidas, remoção de registros com preços inconsistentes e eliminação de duplicatas.  


In [ ]:
# Salvamento Final dos Datasets
print("=== SALVAMENTO DOS DATASETS ===")

# 1. Salvamento do Dataset Limpo (df_clean)
print("\n--- SALVAMENTO DO DATASET LIMPO ---")
print(f"Dataset limpo (df_clean):")
print(f"  Shape: {df_clean.shape}")
print(f"  Valores nulos: {df_clean.isnull().sum().sum()}")
print(f"  Tipos de dados: {dict(df_clean.dtypes.value_counts())}")

output_path_clean = "../database/dataset gerado/dataset_limpo.csv"
df_clean.to_csv(output_path_clean, index=False, encoding="utf-8")

print(f"✅ Dataset limpo exportado com sucesso!")
print(f"Arquivo: {output_path_clean}")

# 2. Salvamento do Dataset Codificado (df_transformed)
print("\n--- SALVAMENTO DO DATASET CODIFICADO ---")
print(f"Dataset codificado (df_transformed):")
print(f"  Shape: {df_transformed.shape}")
print(f"  Valores nulos: {df_transformed.isnull().sum().sum()}")
print(f"  Tipos de dados: {dict(df_transformed.dtypes.value_counts())}")

output_path_transformed = "../database/dataset gerado/dataset_codificado.csv"
df_transformed.to_csv(output_path_transformed, index=False, encoding="utf-8")

print(f"✅ Dataset codificado exportado com sucesso!")
print(f"Arquivo: {output_path_transformed}")

# Resumo final da progressão
print(f"\n=== RESUMO FINAL DA PROGRESSÃO ===")
print(f"df_raw (original):     {df_raw.shape}")
print(f"df_clean (pós-limpeza): {df_clean.shape}")
print(f"df_transformed (final): {df_transformed.shape}")
print(f"\nProcesso concluído: Dados limpos, transformados e salvos!")